In [1]:
import pickle
import plotly.express as px
import pandas as pd
import numpy as np
import torch
import LogLikelihoodTorch 

with open('.DataPhase.pickle','rb') as f:
    X = pickle.load(f)
torch.set_default_dtype(torch.float64)
X_tensor = torch.from_numpy(X).to(torch.float64)


In [2]:
# Setting Up Parameters For Fitting of K Multivariate Watson Distributions

K = 7
p = 90

pi,kappa,mu = LogLikelihoodTorch.Initialize(p=p,K=K)
#%%

learning_rate = 1
n_iters = 3000

#torch.autograd.set_detect_anomaly(False)
Parameters = [
    {'params':pi},
    {'params':kappa},
    {'params':mu}
]

Adam = torch.optim.Adam(Parameters,lr=learning_rate)


In [3]:
# Optimizing Distribution:
Pi_fit,Kappa_fit,mu_fit = LogLikelihoodTorch.Optimizationloop(X_tensor,[pi,kappa,mu],lose=LogLikelihoodTorch.log_likelihood,Optimizer=Adam,n_iters=n_iters,K=K)

epoch 1; Log-Likelihood = -5529658.767288913
epoch 101; Log-Likelihood = -6306845.881398491
epoch 201; Log-Likelihood = -6385702.477403252
epoch 301; Log-Likelihood = -6397880.758594229
epoch 401; Log-Likelihood = -6459081.595185234
epoch 501; Log-Likelihood = -6517219.0344613325
epoch 601; Log-Likelihood = -6546745.744339621
epoch 701; Log-Likelihood = -6574367.353358062
epoch 801; Log-Likelihood = -6607147.772704729
epoch 901; Log-Likelihood = -6647197.5492872605
epoch 1001; Log-Likelihood = -6694035.401104709
epoch 1101; Log-Likelihood = -6747151.734048681
epoch 1201; Log-Likelihood = -6805636.93104538
epoch 1301; Log-Likelihood = -6868862.074130376
epoch 1401; Log-Likelihood = -6935623.012485649
epoch 1501; Log-Likelihood = -6984982.4670212185
epoch 1601; Log-Likelihood = -7020929.010113728
epoch 1701; Log-Likelihood = -7058877.687384576
epoch 1801; Log-Likelihood = -7098496.046222242
epoch 1901; Log-Likelihood = -7139720.418638597
epoch 2001; Log-Likelihood = -7173064.8927161945
e

In [4]:
# Contraining Parametes and Saving Fitted Model
Softmax = torch.nn.Softmax(0)
Softplus = torch.nn.Softplus()

pi_est = Softmax(Pi_fit)
kappa_est = Softplus(Kappa_fit)
mu_est = mu /torch.sqrt((mu_fit * mu_fit).sum(axis=0))

with open('Torch_K=7.pickle','wb') as f:
    pickle.dump([pi_est,kappa_est,mu_est], f)